In [1]:
import re
import nltk
import gensim
import pandas as pd
import numpy as np
import pickle
import random
import theano
import theano.tensor as T
from numpy import random as rng
from sklearn.feature_extraction.text import CountVectorizer
from theano.tensor.shared_randomstreams import RandomStreams
rng = np.random.RandomState(123) 
theano_rng = RandomStreams( rng.randint(2**30)) 

Using gpu device 0: GeForce GT 755M (CNMeM is enabled with initial size: 80.0% of memory, CuDNN not available)


In [2]:
train_ = pd.DataFrame.from_csv('all_train1.csv', index_col=False)
valid_ = pd.DataFrame.from_csv('valid.csv'  , index_col=False)
test_  = pd.DataFrame.from_csv('test.csv'   , index_col=False)


glove_file='corpus_vectors_GloVe_300d.txt'
glove=gensim.models.Word2Vec.load_word2vec_format(glove_file,binary=False)

w2v_file='corpus_vectors_W2V_300d.txt'
w2v=gensim.models.Word2Vec.load_word2vec_format(w2v_file,binary=False)

In [3]:
class Denoising_Autoencoder(object):
    def __init__(self, X, hidden_size, noise_type, noise_level, activation_function, output_function):
        
        self.m = X.shape[0]
        self.n = X.shape[1] - 1
        self.hidden_size = hidden_size
    
        # The clean data
        self.X_clean = X
        self.X_clean = theano.shared(name='X_clean', value=np.asarray(self.X_clean, dtype=theano.config.floatX))
        
        
        # The noisy data
        X_noisy = np.zeros((self.m, self.n+1))
        for i in xrange(self.m):
            for j in xrange(self.n + 1):
                X_noisy[i][j] = X[i][j]
        #X_noisy = X[:,:]
        if noise_type == 'salt_peeper_noise':
            for i in range(self.m):
                random_positions = random.sample(range(0, self.n), int(self.n*noise_level))
                for j in range(int(self.n*noise_level)):
                    X_noisy[i][random_positions[j]] = float(random.getrandbits(1))
        if noise_type == 'mask_noise':
            for i in range(self.m):
                zero_positions = random.sample(range(0, self.n), int(self.n*noise_level))
                for j in range(int(self.n*noise_level)):
                    X_noisy[i][zero_positions[j]] = 0.0
        self.X_noisy = X_noisy
        self.X_noisy = theano.shared(name ='X_noisy', value=np.asarray(self.X_noisy, dtype=theano.config.floatX))
        
        
        
        initial_W = np.asarray(rng.uniform(low = -4 * np.sqrt(6./(self.hidden_size + self.n)),
                                           high = 4 * np.sqrt(6./(self.hidden_size + self.n)),
                                           size = (self.n,self.hidden_size)) , dtype = theano.config.floatX)
        self.W  = theano.shared(name='W', value=initial_W)
        self.b1 = theano.shared(name='b1',value=np.zeros(shape=(self.hidden_size), dtype=theano.config.floatX))
        self.b2 = theano.shared(name='b2',value=np.zeros(shape=(self.n),           dtype=theano.config.floatX))
        
        self.activation_function = activation_function
        self.output_function = output_function
        self.index_list = []                    # During shuffling we need to keep track of word's index
        self.hidden_list = [] 

    
    def trainSDA(self, loss_function, n_epochs = 100, mini_batch_size = 1, learning_rate = 0.1):
        index = T.lscalar()
        x_clean = T.matrix('x_clean')
        x_noisy = T.matrix('x_noisy')
        params = [self.W, self.b1, self.b2]
        
        
        # Forward Propagation
        hidden = self.activation_function(T.dot(x_noisy, self.W) + self.b1)
        output = self.output_function(T.dot(hidden, T.transpose(self.W)) + self.b2)
        
        
        # Loss Expression
        if loss_function == 'cross entropy loss':
            loss = -T.sum(x_clean*T.log(output) + (1-x_clean) * T.log(1-output)).mean()
        if loss_function == 'sum squared loss':
            loss = ((x_clean - output)**2).sum()/(self.m)
        
        
        # Gradient Descent Optimization
        #learning_rate = 0.5
        #updates = []
        #gparams = T.grad(loss,params)
        #for param,gparam in zip(params, gparams):
        #    updates.append((param, param - learning_rate * gparam))
        
        
        # Adam Optimization
        lr=0.0002 
        beta1=0.1 
        beta2=0.001
        epsilon=1e-8 
        gamma=1-1e-7
        updates = []
        grads = theano.grad(loss, params)
        i = theano.shared(np.float32(1))  # HOW to init scalar shared?
        i_t = i + 1.
        fix1 = 1. - (1. - beta1)**i_t
        fix2 = 1. - (1. - beta2)**i_t
        beta1_t = 1-(1-beta1)*gamma**(i_t-1)   # ADDED
        lr_t = lr * (T.sqrt(fix2) / fix1)
        for param_i, g in zip(params, grads):
            m = theano.shared(
                np.zeros(param_i.get_value().shape, dtype=theano.config.floatX))
            v = theano.shared(
                np.zeros(param_i.get_value().shape, dtype=theano.config.floatX))
            m_t = (beta1_t * g) + ((1. - beta1_t) * m) # CHANGED from b_t TO use beta1_t
            v_t = (beta2 * g**2) + ((1. - beta2) * v)
            g_t = m_t / (T.sqrt(v_t) + epsilon)
            param_i_t = param_i - (lr_t * g_t)
            updates.append((m, m_t))
            updates.append((v, v_t))
            updates.append((param_i, param_i_t) )
        updates.append((i, i_t))
        # End of Adam Optimization
        
        
        
        train_func = theano.function(inputs=[index], outputs=[loss], updates=updates,
                            givens={x_clean:self.X_clean[index:index+mini_batch_size,:-1], 
                                    x_noisy:self.X_noisy[index:index+mini_batch_size,:-1]})
        loss_func = theano.function(inputs=[], outputs=[loss],
                                   givens={x_clean:self.X_clean[:,:-1],
                                           x_noisy:self.X_noisy[:,:-1]})
        
        
        
        import time
        start_time = time.clock()
        for epoch in xrange(n_epochs):
            for row in xrange(0, self.m, mini_batch_size):
                train_func(row)
            self.shuffleX()
            if (epoch+1) % 50 == 0:
                print "Epoch : ",epoch+1
                self.index_list.append(self.X_clean[:,-1:].eval())
                self.hidden_list.append(self.get_hidden_layer())
        end_time = time.clock()
        print "Average training timr per epoch : ", (end_time - start_time)/n_epochs
        

        
    def shuffleX(self):
        rng = np.random.RandomState(np.random.randint(0,400)) 
        theano_rng = RandomStreams(rng.randint(2**20)) 
        rows = self.m
        #Z = np.concatenate((self.X_clean, self.X_noisy), axis=1)
        Z = np.concatenate((self.X_clean.eval(), self.X_noisy.eval()), axis=1)
        #Z = theano.shared(name='Z', value=Z)
        Y = T.permute_row_elements(Z.T, theano_rng.permutation(n=rows)).T.eval()
        X_clean_update = (self.X_clean, Y[ :, :self.n+1])
        X_noisy_update = (self.X_noisy, Y[ :, self.n+1:])
        f1 = theano.function(inputs=[],updates=[X_clean_update])
        f2 = theano.function(inputs=[],updates=[X_noisy_update])
        f1()
        f2()
        
   
    def get_hidden_layer(self):
        x = T.matrix('x')
        hidden_approximation = self.activation_function(T.dot(x, self.W) + self.b1)
        hidden_func = theano.function(inputs=[], outputs = hidden_approximation, givens={x:self.X_clean[:,:-1]})
        return hidden_func()
    
    def get_hidden_all(self):
        hidden_all = T.tensor3('hidden_all')
        hidden_all = theano.shared(name='hidden_all', value=self.hidden_list)
        hidden_all_func = theano.function(inputs=[], outputs = hidden_all)
        return hidden_all_func()
    
    def get_index_all(self):
        index_all = T.matrix('index_all')
        index_all = theano.shared(name='index_all', value=self.index_list)
        index_all_func = theano.function(inputs=[], outputs = index_all)
        return index_all_func()

    def get_weights1(self):
        get_weights_func = theano.function(inputs=[], outputs = [self.W, self.b1])
        return get_weights_func()
    
    def get_weights2(self):
        get_weights_func = theano.function(inputs=[], outputs = [T.transpose(self.W), self.b2])
        return get_weights_func()
    
    def get_hidden(self):
        x_clean = T.matrix('x_clean')
        hidden_approximation = self.activation_function(T.dot(x_clean, self.W) + self.b1)
        hidden_func = theano.function(inputs=[], outputs = hidden_approximation, givens={x_clean:self.X_clean})
        return hidden_func()

In [4]:
def review_modification(raw_review):
    letters = re.sub("[^a-zA-Z]", " ", raw_review)
    lower_case = letters.lower()
    words = lower_case.split()
    return(words)

def corpus_review_modification(raw_review):
    letters = re.sub("[^a-zA-Z]", " ", raw_review)
    lower_case = letters.lower()
    words = lower_case.split()
    return( " ".join(words )) 


def create_denoising_autoencoder(X, hidden_size, noise_type, noise_level, n_epoch = 10, batch_size = 20, 
                                 loss_function='sum squared loss', activation_function=T.tanh, output_function=T.tanh):
    
    A = Denoising_Autoencoder(X, hidden_size, noise_type, noise_level, activation_function, output_function)
    A.trainSDA(loss_function, n_epoch, batch_size)
    hidden_all = A.get_hidden_all()
    idx = A.get_index_all()
    return hidden_all, idx
    
# Function to create feature vectors for documents using word-vector averages
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype='float32')
    nwords = 0
    index2word_set = set(model.index2word)
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1.
    return reviewFeatureVecs

In [5]:
# To correct spelling mistakes we can use Peter Norvig's pytohn code
import re, collections
from nltk.corpus import wordnet 
class RepeatReplacer(object): 
    def __init__(self):    
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')    
        self.repl = r'\1\2\3'  
    def replace(self, word):    
        if wordnet.synsets(word):      
            return word    
        repl_word = self.repeat_regexp.sub(self.repl, word)    
        if repl_word != word:      
            return self.replace(repl_word)    
        else:      
            return repl_word
        

def words(text): return re.findall('[a-z]+', text.lower()) 

def train_spelling(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train_spelling(words(file('big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)
replacer = RepeatReplacer()

In [6]:
corpus = []
clean_train_reviews = []
clean_valid_reviews = []
clean_test_reviews  = []
len_train = train_['sent'].size
len_valid = valid_['sent'].size
len_test  = test_['sent'].size

for i in xrange(len_train):
    clean_train_reviews.append(review_modification(train_['sent'][i]))
    corpus.append(corpus_review_modification(train_['sent'][i]))
for i in xrange(len_valid):
    clean_valid_reviews.append(review_modification(valid_['sent'][i]))
    corpus.append(corpus_review_modification(valid_['sent'][i]))
for i in xrange(len_test):
    clean_test_reviews.append(review_modification(test_['sent'][i]))
    corpus.append(corpus_review_modification(test_['sent'][i]))
    
vectorizer = CountVectorizer(analyzer = 'word', tokenizer = None, preprocessor = None, stop_words = None)
vectorizer.fit_transform(corpus)
vocabulary = vectorizer.vocabulary_
index = vocabulary.values()
index = np.array(index).reshape((len(vocabulary),1))
voc_words = vocabulary.keys()
print len(vocabulary) 
mapping = dict((v,k) for k,v in vocabulary.iteritems())

16974


In [7]:
my_vector1 = np.random.randn(16974,300)
for i in xrange(16974):
    try:
        my_vector1[i] = glove[voc_words[i]]
    except KeyError:
        try:
            my_vector1[i] = glove[correct(replacer.replace(voc_words[i]))]
        except KeyError:
            my_vector1[i] = np.random.uniform(low=-0.25, high=0.25, size=(1,300))

In [8]:
my_vector2 = np.random.randn(16974,300)
for i in xrange(16974):
    try:
        my_vector2[i] = w2v[voc_words[i]]
    except KeyError:
        try:
            my_vector2[i] = w2v[correct(replacer.replace(voc_words[i]))]
        except KeyError:
            my_vector2[i] = np.random.uniform(low=-0.25, high=0.25, size=(1,300))

In [9]:
my_vector = np.concatenate((my_vector1, my_vector2, index),axis=1)

In [10]:
print 'For no. of epochs = 300'
X_hidden, index = create_denoising_autoencoder(my_vector, noise_type = 'mask_noise', noise_level=0.35, 
                                               hidden_size = 300, n_epoch = 300)

For no. of epochs = 300
Epoch :  50
Epoch :  100
Epoch :  150
Epoch :  200
Epoch :  250
Epoch :  300
Average training timr per epoch :  2.85564898096


In [11]:
X_hidden = np.array(X_hidden)
index = np.array(index)
index = index.reshape((6, 16974))
with open('X_hidden35.data', 'w') as f:
    pickle.dump(X_hidden, f)
with open('index35.data', 'w') as f:
    pickle.dump(index, f)
with open('mapping.data', 'w') as f:
    pickle.dump(mapping, f)

In [6]:
with open('clean_train_reviews.data', 'w') as f:
    pickle.dump(clean_train_reviews, f)
with open('clean_valid_reviews.data', 'w') as f:
    pickle.dump(clean_valid_reviews, f)
with open('clean_test_reviews.data', 'w') as f:
    pickle.dump(clean_test_reviews, f)